Django Installation

In [ ]:
# Recommended: create a virtual environment first
python -m venv myenv
source myenv/bin/activate  # On Windows: myenv\Scripts\activate

# Install Django
pip install django

# Verify installation
python -m django --version

Django folder structure and flow of control

In [ ]:
myproject/
    manage.py
    myproject/
        __init__.py
        settings.py
        urls.py
        asgi.py
        wsgi.py
    myapp/
        migrations/
        __init__.py
        admin.py
        apps.py
        models.py
        tests.py
        views.py

Creation of first Django project

In [ ]:
# Create project
django-admin startproject myproject

# Create app
cd myproject
python manage.py startapp myapp

# Add app to INSTALLED_APPS in myproject/settings.py
INSTALLED_APPS = [
    ...
    'myapp',
]

# Run development server
python manage.py runserver

# Django models

Models define the structure of your database:

In [ ]:
# myapp/models.py
from django.db import models

class Product(models.Model):
    name = models.CharField(max_length=100)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    description = models.TextField()
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    def __str__(self):
        return self.name

After creating models:

In [ ]:
# Create migrations
python manage.py makemigrations

# Apply migrations
python manage.py migrate

# Django admin

Django provides an automatic admin interface:

In [ ]:
# myapp/admin.py
from django.contrib import admin
from .models import Product

admin.site.register(Product)

Create superuser:

In [ ]:
python manage.py createsuperuser

# Django urls

URL routing connects URLs to views:

In [ ]:
# myproject/urls.py
from django.contrib import admin
from django.urls import path
from myapp import views

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', views.home, name='home'),
    path('products/', views.product_list, name='product_list'),
]

# Django views

In [ ]:
# myapp/views.py
from django.shortcuts import render
from .models import Product

def home(request):
    return render(request, 'home.html')

def product_list(request):
    products = Product.objects.all()
    return render(request, 'products/list.html', {'products': products})

Django settings
Key settings in settings.py:

DEBUG: True in development, False in production

ALLOWED_HOSTS: Hosts that can serve the application

INSTALLED_APPS: Apps included in the project

DATABASES: Database configuration

STATIC_URL: URL prefix for static files

MEDIA_URL: URL prefix for user-uploaded files

# Django migrations

Migrations track changes to your models:

In [ ]:
# After changing models
python manage.py makemigrations

# To apply migrations
python manage.py migrate

# To see SQL for a migration
python manage.py sqlmigrate myapp 0001

# To squash migrations (combine multiple into one)
python manage.py squashmigrations myapp 0001 0005

# Coding Examples

Basic Django Project Setup (Easy)

In [ ]:
# Create project and app
django-admin startproject bookstore
cd bookstore
python manage.py startapp catalog

# Add to INSTALLED_APPS in bookstore/settings.py
INSTALLED_APPS = [
    ...
    'catalog',
]

# Create first model
# catalog/models.py
from django.db import models

class Book(models.Model):
    title = models.CharField(max_length=200)
    author = models.CharField(max_length=100)
    price = models.DecimalField(max_digits=6, decimal_places=2)
    
    def __str__(self):
        return self.title

# Create and apply migrations
python manage.py makemigrations
python manage.py migrate

# Set up admin
# catalog/admin.py
from django.contrib import admin
from .models import Book

admin.site.register(Book)

# Create superuser
python manage.py createsuperuser

Views and URLs (Medium)

In [ ]:
# catalog/views.py
from django.shortcuts import render
from .models import Book

def book_list(request):
    books = Book.objects.all()
    return render(request, 'catalog/book_list.html', {'books': books})

def book_detail(request, pk):
    book = Book.objects.get(pk=pk)
    return render(request, 'catalog/book_detail.html', {'book': book})

# bookstore/urls.py
from django.contrib import admin
from django.urls import path
from catalog import views

urlpatterns = [
    path('admin/', admin.site.urls),
    path('books/', views.book_list, name='book_list'),
    path('books/<int:pk>/', views.book_detail, name='book_detail'),
]

# Create templates in catalog/templates/catalog/
# book_list.html
{% for book in books %}
    <h2><a href="{% url 'book_detail' book.pk %}">{{ book.title }}</a></h2>
    <p>Author: {{ book.author }}</p>
    <p>Price: ${{ book.price }}</p>
{% endfor %}

# book_detail.html
<h1>{{ book.title }}</h1>
<p>Author: {{ book.author }}</p>
<p>Price: ${{ book.price }}</p>
<a href="{% url 'book_list' %}">Back to list</a>

Forms and ModelForms (Medium-Hard)

In [ ]:
# catalog/forms.py
from django import forms
from .models import Book

class BookForm(forms.ModelForm):
    class Meta:
        model = Book
        fields = ['title', 'author', 'price']
        widgets = {
            'title': forms.TextInput(attrs={'class': 'form-control'}),
            'author': forms.TextInput(attrs={'class': 'form-control'}),
            'price': forms.NumberInput(attrs={'class': 'form-control'}),
        }

# catalog/views.py
from django.shortcuts import render, redirect
from .forms import BookForm

def book_create(request):
    if request.method == 'POST':
        form = BookForm(request.POST)
        if form.is_valid():
            form.save()
            return redirect('book_list')
    else:
        form = BookForm()
    return render(request, 'catalog/book_form.html', {'form': form})

# catalog/templates/catalog/book_form.html
<form method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit" class="btn btn-primary">Save</button>
</form>

# Add URL pattern
path('books/new/', views.book_create, name='book_create'),

Class-Based Views (Hard)

In [ ]:
# catalog/views.py
from django.views.generic import ListView, DetailView, CreateView, UpdateView, DeleteView
from django.urls import reverse_lazy
from .models import Book

class BookListView(ListView):
    model = Book
    template_name = 'catalog/book_list.html'
    context_object_name = 'books'

class BookDetailView(DetailView):
    model = Book
    template_name = 'catalog/book_detail.html'

class BookCreateView(CreateView):
    model = Book
    form_class = BookForm
    template_name = 'catalog/book_form.html'
    success_url = reverse_lazy('book_list')

class BookUpdateView(UpdateView):
    model = Book
    form_class = BookForm
    template_name = 'catalog/book_form.html'
    success_url = reverse_lazy('book_list')

class BookDeleteView(DeleteView):
    model = Book
    template_name = 'catalog/book_confirm_delete.html'
    success_url = reverse_lazy('book_list')

# Update URLs
path('books/', BookListView.as_view(), name='book_list'),
path('books/<int:pk>/', BookDetailView.as_view(), name='book_detail'),
path('books/new/', BookCreateView.as_view(), name='book_create'),
path('books/<int:pk>/edit/', BookUpdateView.as_view(), name='book_update'),
path('books/<int:pk>/delete/', BookDeleteView.as_view(), name='book_delete'),

REST API with Django REST Framework (Hard)

In [ ]:
# Install DRF
pip install djangorestframework

# Add to INSTALLED_APPS
'rest_framework',

In [ ]:
# catalog/serializers.py
from rest_framework import serializers
from .models import Book

class BookSerializer(serializers.ModelSerializer):
    class Meta:
        model = Book
        fields = ['id', 'title', 'author', 'price']

# catalog/views.py
from rest_framework import generics
from .serializers import BookSerializer

class BookListCreate(generics.ListCreateAPIView):
    queryset = Book.objects.all()
    serializer_class = BookSerializer

class BookRetrieveUpdateDestroy(generics.RetrieveUpdateDestroyAPIView):
    queryset = Book.objects.all()
    serializer_class = BookSerializer

# catalog/urls.py
from django.urls import path
from .views import BookListCreate, BookRetrieveUpdateDestroy

urlpatterns = [
    path('api/books/', BookListCreate.as_view()),
    path('api/books/<int:pk>/', BookRetrieveUpdateDestroy.as_view()),
]

# Include in project URLs
path('catalog/', include('catalog.urls')),